In [1]:
# Base python imports
import os
import sys
import time
import datetime

# Assuming the notebook is located in <ROOT>/notebooks
sys.path.append(os.path.abspath('..'))

#PyPi imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from dotenv import load_dotenv

load_dotenv()

# Custom imports
from src.modeling.utils.OpticalFlowUtils import FlowReader
from src.modeling.utils.OpticalFlowUtils import FlowVisualiser
from src.modeling.utils.MPISintelUtils import MPISintelHandler
from src.modeling.customML.customLosses.EndPointErrorLoss import AEPE_Loss
from src.modeling.customML.customModels.OpticalFlowUnrolling import UnrolledOFModel

2024-09-29 12:44:13.422983: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-29 12:44:15.032932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 12:44:15.713759: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 12:44:15.716929: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 12:44:16.825095: I tensorflow/core/platform/cpu_feature_gua

In [2]:
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.16.2


In [3]:
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU(s): {gpus}")
if gpus:
    # Restrict TensorFlow to only allocate 8.1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=float(os.environ.get("CUDA_MEMORY_LIMIT_MB")))])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
        print(f"GPU(s): {gpus}")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(f"Runtime Error: {e}")

GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs
GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-09-29 12:44:35.447004: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-29 12:44:36.975136: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-29 12:44:36.975168: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-29 12:44:36.977749: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-29 12:44:36.977777: I external/local_xla/xla/stream_executor

In [4]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [5]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
reader = FlowReader()
visualiser = FlowVisualiser()
dataHandler = MPISintelHandler()

In [7]:
RAW_DATA_PATH = os.path.join("..", "data", "raw")
PROCESSED_DATA_PATH = os.path.join("..", "data", "processed")

IMAGES_PATH = os.path.join(PROCESSED_DATA_PATH, "trainImages.data")
FLOWS_PATH = os.path.join(PROCESSED_DATA_PATH, "trainFlows.data")

trainImgsArray = dataHandler.loadData(IMAGES_PATH)
trainFlowsArray = dataHandler.loadData(FLOWS_PATH)

print(f"Sintel images shape: {trainImgsArray.shape}")
print(f"Sintel flows shape: {trainFlowsArray.shape}")

Sintel images shape: (1041, 2, 436, 1024)
Sintel flows shape: (1041, 436, 1024, 2)


In [8]:
TRAIN_IMAGES_PATH = os.path.join(PROCESSED_DATA_PATH, "trainX.data")
TRAIN_FLOWS_PATH = os.path.join(PROCESSED_DATA_PATH, "trainy.data")

train_X = dataHandler.loadData(TRAIN_IMAGES_PATH)
train_X = train_X.astype('float32') / 255.0
train_X = tf.reshape(train_X, shape=(-1, 2, 436, 1024, 1))

train_y = dataHandler.loadData(TRAIN_FLOWS_PATH)
train_y = train_y.astype('float32')

TEST_IMAGES_PATH = os.path.join(PROCESSED_DATA_PATH, "testX.data")
TEST_FLOWS_PATH = os.path.join(PROCESSED_DATA_PATH, "testy.data")

test_X = dataHandler.loadData(TEST_IMAGES_PATH)
test_X = test_X.astype('float32') / 255.0
test_X = tf.reshape(test_X, shape=(-1, 2, 436, 1024, 1))

test_y = dataHandler.loadData(TEST_FLOWS_PATH)
test_y = test_y.astype('float32')

VAL_IMAGES_PATH = os.path.join(PROCESSED_DATA_PATH, "valX.data")
VAL_FLOWS_PATH = os.path.join(PROCESSED_DATA_PATH, "valy.data")

val_X = dataHandler.loadData(VAL_IMAGES_PATH)
val_X = val_X.astype('float32') / 255.0
val_X = tf.reshape(val_X, shape=(-1, 2, 436, 1024, 1))

val_y = dataHandler.loadData(VAL_FLOWS_PATH)
val_y = val_y.astype('float32')

In [9]:
print(f"train_X: {train_X.shape} test_X: {test_X.shape} val_X: {val_X.shape} -> Total Samples: {train_X.shape[0] + test_X.shape[0] + val_X.shape[0]}")
print(f"train_y: {train_y.shape} test_y: {test_y.shape} val_y: {val_y.shape} -> Total Samples: {train_y.shape[0] + test_y.shape[0] + val_y.shape[0]}")

train_X: (510, 2, 436, 1024, 1) test_X: (297, 2, 436, 1024, 1) val_X: (234, 2, 436, 1024, 1) -> Total Samples: 1041
train_y: (510, 436, 1024, 2) test_y: (297, 436, 1024, 2) val_y: (234, 436, 1024, 2) -> Total Samples: 1041


In [10]:
model = UnrolledOFModel(num_iterations = 5, numPyramidLevels=2, correlationMaxDisplacement = 2, name = "UnrollingOF")
model.summary()

Model: "UnrollingOF"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cost_volume_correlation_layer   │ ?                      │   0 (unbuilt) │
│ (CostVolumeCorrelationLayer)    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ImagePyramidLayer               │ ?                      │   0 (unbuilt) │
│ (ImagePyramidLayer)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
log_dir = os.path.join("..", "assets", "logs", "fits", f"{model.name}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_images=True)

modelDir = os.path.join("..", "assets", "ml", "models", model.name)
if not os.path.exists(modelDir):
    os.makedirs(modelDir)

modelPath = os.path.join(modelDir, f"{model.name}_best.weights.h5")
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=modelPath, monitor='val_loss', save_best_only=True, save_weights_only=True)

learning_rate_callback = tf.keras.callbacks.ReduceLROnPlateau(factor=0.85, patience=50, min_lr=1e-6)
_callbacks = [tensorboard_callback, model_checkpoint_callback, learning_rate_callback]

callbacks = tf.keras.callbacks.CallbackList(_callbacks, add_history=False)

fileWriter = tf.summary.create_file_writer(os.path.join(log_dir, "metrics"))
fileWriter.set_as_default()

In [12]:
if os.path.isfile(modelPath):
    print(f"Loading model from path: {modelPath}")
    model.load_weights(modelPath)
else:
    print("No model to load")

No model to load


In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999)

In [14]:
model.compile(optimizer=optimizer, loss=AEPE_Loss(), metrics = ['mse'])
# TODO: Fix the below implementation for the older version of tensorflow.
# Issue appears with the building with the batch size being of 'None' type.
model.build(input_shape=(None, 2, 436, 1024, 1))
model.summary()

/mnt/a/OneDrive/Repos/PIOFE-Unrolling/.venv-linux/lib/python3.10/site-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'UnrollingOF', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "UnrollingOF"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cost_volume_correlation_layer   │ ?                      │   0 (unbuilt) │
│ (CostVolumeCorrelationLayer)    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ImagePyramidLayer               │ ?                      │   0 (unbuilt) │
│ (ImagePyramidLayer)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:
tf.compat.v1.trainable_variables(model)

[]

In [16]:
hist = model.fit(x=train_X, y=train_y, epochs = 15, verbose = True, batch_size=1, validation_data=(test_X, test_y), callbacks=_callbacks)

Epoch 1/15


I0000 00:00:1727585107.705370    7607 service.cc:145] XLA service 0x7fbfdc014c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727585107.705410    7607 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-09-29 12:45:08.767887: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-29 12:45:09.290367: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-09-29 12:45:10.342408: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  %slice.242 = f32[1,1,436,1024,1]{4,3,2,1,0} slice(f32[1,2,436,1024,1]{4,3,2,1,0} %constant.241), slice={[0:1], [0:1], [0:436], [0:1024], [0:1]}, metadata={op_type="StridedSlice" op_name="UnrollingOF_1/StatefulPartitionedCall/strided_slice" source_file="/m

  1/510 ━━━━━━━━━━━━━━━━━━━━ 44:30:19 315s/step - loss: 1.0404 - mse: 5.1873

I0000 00:00:1727585731.622196    7606 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_select_fusion_1', 14240 bytes spill stores, 18640 bytes spill loads



  7/510 ━━━━━━━━━━━━━━━━━━━━ 44:28:46 318s/step - loss: 21.2712 - mse: 902.4386

2024-09-29 13:27:26.575950: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1566] failed to allocate 54.50MiB (57147392 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-09-29 13:27:26.576183: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INTERNAL: Failed to allocate 57147392 bytes for new constant
	 [[{{node StatefulPartitionedCall}}]]


InternalError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/mnt/a/OneDrive/Repos/PIOFE-Unrolling/.venv-linux/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/mnt/a/OneDrive/Repos/PIOFE-Unrolling/.venv-linux/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/mnt/a/OneDrive/Repos/PIOFE-Unrolling/.venv-linux/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/mnt/a/OneDrive/Repos/PIOFE-Unrolling/.venv-linux/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/ev

In [22]:
def visualiseFits(fits, validationData = True):
    fig, axs = plt.subplots(2, 2, figsize = (40, 20))

    axs[0][0].plot(fits.history['loss'], label = "Conventional NN Train EPE")
    axs[0][0].legend()
    axs[0][0].set_xlabel("Epochs")
    axs[0][0].set_ylabel("EPE (log Scaled)")
    axs[0][0].set_yscale('log')
    axs[0][0].grid(True)

    axs[1][0].plot(fits.history['loss'], label = "Conventional NN Train EPE")
    axs[1][0].legend()
    axs[1][0].set_xlabel("Epochs")
    axs[1][0].set_ylabel("EPE")
    axs[1][0].grid(True)

    axs[0][1].plot(fits.history['mse'], label = "Conventional NN Train MSE")
    axs[0][1].legend()
    axs[0][1].set_xlabel("Epochs")
    axs[0][1].set_ylabel("MSE")
    axs[0][1].grid(True)

    axs[1][1].plot(fits.history['mse'], label = "Conventional NN Train MSE")
    axs[1][1].legend()
    axs[1][1].set_xlabel("Epochs")
    axs[1][1].set_ylabel("MSE (log scaled)")
    axs[1][1].set_yscale('log')
    axs[1][1].grid(True)
    
    if validationData:
        axs[0][0].plot(fits.history['val_loss'], label = "Conventional NN Train EPE validation")
        axs[1][0].plot(fits.history['val_loss'], label = "Conventional NN Train EPE validation")
        axs[0][1].plot(fits.history['val_mse'], label = "Conventional NN Train MSE validation")
        axs[1][1].plot(fits.history['val_mse'], label = "Conventional NN Train MAE validation")

    plt.show()

In [ ]:
visualiseFits(hist)

In [ ]:
model.summary()

In [ ]:
model.alphas

In [ ]:
result = model(train_X[142].numpy().reshape(1, 2, 436, 1024, 1))
imgs = train_X[142].numpy()
result[0].shape

In [ ]:
imgs.shape

In [ ]:
result[0].numpy()

In [ ]:
visualiser.visualiseOpticalFlow(result[0].numpy(), drawArrows=True, step=10, scale=1)
visualiser.visualiseOpticalFlow(result[0].numpy(), drawArrows=False, step=10, scale=1)
visualiser.visualiseOpticalFlow(train_y[142], drawArrows=True, step=10, scale = 1)
visualiser.visualiseFlowError(groundTruth=train_y[142], predicted=result[0].numpy())
plt.imshow(imgs[0], cmap = 'gray')
plt.show()
plt.imshow(imgs[1], cmap = 'gray')
plt.show()

In [21]:
SAMPLE_INDEX = 69

images = train_X[SAMPLE_INDEX].numpy()
img1 = images[0]
img2 = images[1]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (15, 30))
ax[0].imshow(img1, cmap='gray')
ax[1].imshow(img2, cmap='gray')
plt.show()

In [ ]:
result = model(train_X[SAMPLE_INDEX].numpy().reshape(1, 2, 436, 1024, 1))
visualiser.visualiseOpticalFlow(result[0].numpy(), drawArrows=True, step=10, scale=1)
visualiser.visualiseOpticalFlow(train_y[SAMPLE_INDEX], drawArrows=True, step=10, scale = 1)
visualiser.visualiseFlowError(groundTruth=train_y[SAMPLE_INDEX], predicted=result[0].numpy())

In [ ]:
import tensorflow_addons as tfa

warped = tfa.image.dense_image_warp(img2.reshape(1, img2.shape[0], img2.shape[1], img2.shape[2]), -train_y[SAMPLE_INDEX])
warped.shape

In [ ]:
plt.imshow(warped.numpy()[0], cmap = 'gray')

In [40]:
import cv2

def warp_flow(flow, img1=None, img2=None, interpolation=cv2.INTER_LINEAR):
    """Use remap to warp flow, generating a new image. 
Args:
    flow (np.ndarray): flow
    img1 (np.ndarray, optional): previous frame
    img2 (np.ndarray, optional): next frame
Returns:
    warped image
If img1 is input, the output will be img2_warped, but there will be multiple pixels corresponding to a single pixel, resulting in sparse holes. 
If img2 is input, the output will be img1_warped, and there will be no sparse holes. The latter approach is preferred.
    """
    h, w, _ = flow.shape
    remap_flow = flow.transpose(2, 0, 1)
    remap_xy = np.float32(np.mgrid[:h, :w][::-1])
    if img1 is not None:
        uv_new = (remap_xy + remap_flow).round().astype(np.int32)
        mask = (uv_new[0] >= 0) & (uv_new[1] >= 0) & (uv_new[0] < w) & (uv_new[1] < h)
        uv_new_ = uv_new[:, mask]
        remap_xy[:, uv_new_[1], uv_new_[0]] = remap_xy[:, mask]
        remap_x, remap_y = remap_xy
        img2_warped = cv2.remap(img1, remap_x, remap_y, interpolation)
        mask_remaped = np.zeros((h, w), np.bool8)
        mask_remaped[uv_new_[1], uv_new_[0]] = True
        img2_warped[~mask_remaped] = 0
        return img2_warped
    elif img2 is not None:
        remap_x, remap_y = np.float32(remap_xy + remap_flow)
        return cv2.remap(img2, remap_x, remap_y, interpolation)

In [41]:
opencvwarp = warp_flow(flow=train_y[SAMPLE_INDEX], img2=img2)

In [ ]:
plt.imshow(opencvwarp, cmap='gray')

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = (30, 30))
ax[0, 0].imshow(img1, cmap='gray')
ax[0, 1].imshow(img2, cmap='gray')
ax[1, 0].imshow(warped.numpy()[0], cmap='gray')
ax[1, 1].imshow(warped.numpy()[0] - opencvwarp.reshape(opencvwarp.shape[0], opencvwarp.shape[1], 1), cmap='gray')

plt.tight_layout()
plt.show()